# My first project 

When creating a project, we start defining the infrastructure layout via two files:

  - ansible.cfg
  - inventory
  
Our current setup is:

  - direct access to client machines
  - one `inventory` file
  - no host_key_check
  - no retry files
  - client machine username/password is `root:root`
  - no pki certificate is installed by default



In [1]:
cd exercise-01/


/notebooks/exercise-01


## Preparation

The first steps include creating all secret files, the deployment descriptor and the inventory.

In [3]:
# At first create a proper ssh key for the project
!ssh-keygen -q -t ecdsa -f id_ansible  -N ''

# We'll reference this secret

id_ansible already exists.
Overwrite (y/n)? ^C


In [36]:

# Create a configurationfile. 
with open("ansible.cfg", "wb") as fh:
    fh.write("""
[defaults]
inventory = inventory
retry_files_enabled = false

#
# ssh configs
#
host_key_checking = no
private_key_file = id_ansible

# We want to store root password in an encrypted file
#  but don't want to be prompted for a password each time
vault_password_file = .pin
""")
    
    
# Now a `secret` password ;)
!echo secret > .pin    


# then a small inventory
with open("inventory", "wb") as fh:
    fh.write("""
[course]
172.17.0.[2:4]  ansible_password={{root_password}}  # load variable from vault

""")

    
#
# Check the outcome
#
!cat ansible.cfg inventory



[defaults]
inventory = inventory
retry_files_enabled = false

#
# ssh configs
#
host_key_checking = no
private_key_file = id_ansible

# We want to store root password in an encrypted file
#  but don't want to be prompted for a password each time
vault_password_file = .pin

[course]
172.17.0.[2:4]  ansible_password={{root_password}}  # load variable from vault



In [8]:
# Let's get an error: root_password is UNDEFINED. 
!ansible -m ping all


172.17.0.2 | FAILED! => {
    "failed": true, 
    "msg": "the field 'password' has an invalid value, which appears to include a variable that is undefined. The error was: 'root_password' is undefined"
}
172.17.0.3 | FAILED! => {
    "failed": true, 
    "msg": "the field 'password' has an invalid value, which appears to include a variable that is undefined. The error was: 'root_password' is undefined"
}
172.17.0.4 | FAILED! => {
    "failed": true, 
    "msg": "the field 'password' has an invalid value, which appears to include a variable that is undefined. The error was: 'root_password' is undefined"
}


## Vaults

We now ensure that everything is encrypted and we're able to connect to some hosts

In [16]:
# A vault is just a yaml file containing a dictionary of secrets.
#  We can put here as many information as we want, but for now
#  just put the `root_password`.

!echo "root_password: root" > vaultfile.yml

# We need to encrypt it 
!ansible-vault encrypt vaultfile.yml

Encryption successful


In [17]:
# And show the anatomy of the vault.
!cat vaultfile.yml

# Can we decrypt it?
!ansible-vault view vaultfile.yml

$ANSIBLE_VAULT;1.1;AES256
64623339356534653931373637656137363432663764633834613338343066636165396438633365
3830393863346533633065623339343939616362323366660a356661333037386164663935313165
39396236343666643966356634623362303234636264653939346534666461323265373137623132
6361393763313232390a373631353431313466363765363538633963383864386630646632366633
30343164643733636431643739656630383230636234633234646362643930313733
root_password: root


In [19]:
!ansible -m ping all -e@vaultfile.yml


172.17.0.3 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.3 port 22: Connection refused\r\n", 
    "unreachable": true
}
172.17.0.4 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.4 port 22: No route to host\r\n", 
    "unreachable": true
}
172.17.0.2 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## Our first playbook

We want to switch from password to ssh authentication. Create a playbook to install ssh keys.


In [33]:
with open("copy-key.yml", "wb") as fh:
    fh.write("""
# Load variables from the following files
- hosts: course
  vars_files:
  - vaultfile.yml
  tasks:
  - name: Ensure /root/.ssh is present
    file: path=/root/.ssh owner=root group=root state=directory
  - name: Copy there my key
    copy: src=id_ansible.pub dest=/root/.ssh/authorized_keys mode=0600 owner=root group=root
    
    """)

!ansible-playbook copy-key.yml


PLAY [course] ******************************************************************

TASK [Gathering Facts] *********************************************************
fatal: [172.17.0.3]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.3 port 22: Connection refused\r\n", "unreachable": true}
fatal: [172.17.0.4]: UNREACHABLE! => {"changed": false, "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.4 port 22: No route to host\r\n", "unreachable": true}
ok: [172.17.0.2]

TASK [Ensure /root/.ssh is present] ********************************************
ok: [172.17.0.2]

TASK [Copy there my key] *******************************************************
ok: [172.17.0.2]

PLAY RECAP *********************************************************************
172.17.0.2                 : ok=3    changed=0    unreachable=0    failed=0   
172.17.0.3                 : ok=0    changed=0    unreachable=1    failed=0   
172

In [38]:
# Now comment out the ansible_password field in `inventory` here
http://0.0.0.0:8888/edit/notebooks/notebooks/exercise-01/inventory#

# Running in debug mode we can see all ssh arguments injected via anisble. Discuss the DEBUG output       
!ansible -vvv -m ping all --limit 172.17.0.2

Using /notebooks/notebooks/exercise-01/ansible.cfg as config file
META: ran handlers
Using module file /usr/local/lib/python2.7/site-packages/ansible/modules/system/ping.py
<172.17.0.2> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.2> SSH: EXEC ssh -C -o ControlMaster=auto -o ControlPersist=60s -o StrictHostKeyChecking=no -o 'IdentityFile="id_ansible"' -o KbdInteractiveAuthentication=no -o PreferredAuthentications=gssapi-with-mic,gssapi-keyex,hostbased,publickey -o PasswordAuthentication=no -o ConnectTimeout=10 -o ControlPath=/root/.ansible/cp/e1be7488c1 172.17.0.2 '/bin/sh -c '"'"'echo ~ && sleep 0'"'"''
<172.17.0.2> (0, '/root\n', '')
<172.17.0.2> ESTABLISH SSH CONNECTION FOR USER: None
<172.17.0.2> SSH: EXEC ssh -C -o ControlMaster=auto -o ControlPersist=60s -o StrictHostKeyChecking=no -o 'IdentityFile="id_ansible"' -o KbdInteractiveAuthentication=no -o PreferredAuthentications=gssapi-with-mic,gssapi-keyex,hostbased,publickey -o PasswordAuthentication=no -o ConnectTimeout=10 -o 

# Discuss vaults for:
- common secrets to be shared on a repo
- private secrets to reside on local PC